   En vista de que en el metodo K-means el valor de k esta perdido, usaremos el metodo de ISODATA para que calcule el numero de clusters cercano al optimo

In [1]:
# Inicializar el ambiente
import numpy as np
import pandas as pd
import math
import random
import time
import sys
from scipy.spatial import distance
np.set_printoptions(precision=2, suppress=True) # Cortar la impresión de decimales a 1

LARGER_DISTANCE = sys.maxsize
TALK = True # TALK = True, imprime resultados parciales

In [3]:
# Leer los datos de archivo
df = pd.read_csv("numeric_vgsales.csv")
print (df.describe())

DATA_SET = df.values
DATA_LEN = len(DATA_SET)

# Definir una clase para expresar puntos y su asignación a un cluster
class DataPoint:
    def __init__(self, p):
        self.value = p[:]
        
    def set_value(self, p):
        self.value = p
    
    def get_value(self):
        return self.value
    
    def set_cluster(self, cluster):
        self.cluster = cluster
    
    def get_cluster(self):
        return self.cluster

data = []
def initialize_dataset():
    for i in range(DATA_LEN):
        point = DataPoint(DATA_SET[i])
        point.set_cluster(None)
        data.append(point)
    return

# --------------------------
# Crear el conjunto de datos
initialize_dataset()

         Unnamed: 0          Name      Platform          Year         Genre  \
count  11861.000000  11861.000000  11861.000000  11861.000000  11861.000000   
mean    5930.000000      0.007734      0.007835      0.009182      0.008000   
std     3424.120106      0.004950      0.004787      0.000026      0.004507   
min        0.000000      0.000002      0.000873      0.009063      0.001347   
25%     2965.000000      0.003465      0.004365      0.009168      0.004041   
50%     5930.000000      0.007187      0.006111      0.009186      0.008082   
75%     8895.000000      0.011804      0.011349      0.009200      0.012124   
max    11860.000000      0.017449      0.021824      0.009246      0.016165   

          Publisher      NA_Sales         Exito  
count  11861.000000  11861.000000  11861.000000  
mean       0.004891      0.003308      0.252087  
std        0.007771      0.008566      0.434229  
min        0.000120      0.000091      0.000000  
25%        0.000720      0.000544     

In [4]:
K_INIT = 5
N_MIN = 3
I_MAX = 10
S_MAX = 5
L_MIN = 80
P_MAX = 2

num_clusters = K_INIT # valor de k
iteration = 0

In [5]:
# Definir forma de muestreo; 0 = random, 1=head, 2=tail
SAMPLING_METHOD = 1 

centroids = []
def initialize_centroids():
    if (TALK) : 
        print("Centroides inicializados en:")
    for c in range(num_clusters):
        if (SAMPLING_METHOD == 0) :
            which = random.randint(0,DATA_LEN-1)
        elif (SAMPLING_METHOD == 1):
            which = c
        else :
            which = DATA_LEN-1 - c
                
        centroids.append(list(DATA_SET[which]))
        if (TALK) : 
            print(centroids[c])        
    if (TALK) : 
        print()
    
    return

# --------------------------
# Inicializar los centroides
initialize_centroids()

Centroides inicializados en:
[0.0, 0.017101840543094545, 0.0078568018884454544, 0.0091680360032164645, 0.0026941311321977261, 0.01823301610861721, 9.0654365749241037e-05, 0.0]
[1.0, 0.0060136355869593282, 0.016586581764495959, 0.0091680360032164645, 0.0094294589626920406, 0.0059977026673082925, 9.0654365749241037e-05, 0.0]
[2.0, 0.017449475633331685, 0.0078568018884454544, 0.0091634588509432648, 0.0026941311321977261, 0.02015228096215586, 9.0654365749241037e-05, 0.0]
[3.0, 0.01744721826261586, 0.0034919119504202021, 0.0091909217645824558, 0.010776524528790904, 0.016553659361770886, 9.0654365749241037e-05, 0.0]
[4.0, 0.015393010911214587, 0.013094669814075756, 0.0091726131554896624, 0.01481772122708749, 0.000719724320076995, 9.0654365749241037e-05, 0.0]



In [6]:
elim = 0
members = []
def update_clusters():
    global num_clusters, elim, members
    changed = False
    
    if (TALK) :
        print("Actualizando clusters")
    for i in range(DATA_LEN):
        minDistance = LARGER_DISTANCE
        currentCluster = 0
        
        for j in range(num_clusters):
            dist = distance.euclidean(data[i].get_value(), centroids[j])
            if(dist < minDistance):
                minDistance = dist
                currentCluster = j
        
        if(data[i].get_cluster() is None or data[i].get_cluster() != currentCluster):
            data[i].set_cluster(currentCluster)
            changed = True
            
    members = [0] * num_clusters
    for i in range(DATA_LEN):
        members[data[i].get_cluster()] += 1
    
    if (TALK) : 
        for j in range(num_clusters):
            print("El cluster ", j, " incluye ", members[j], "miembros.")
        print()

    # Marcar los grupos a eliminar      
    to_eliminate = []
    for j in range(num_clusters):
        if (members[j] < N_MIN):
            to_eliminate.append(j)
                
    # Eliminar grupos marcados y recorrer índices
    elim = 0
    is_null = 0
    for j in to_eliminate: # to_eliminate contiene los índices originales a eliminar...
        if (TALK) :
            print("Eliminando cluster ", j)
        # Puesto que se han eliminado 'elim' clusters, los índices se han recorrido 
        # 'elim' veces; el verdero ínidice a eliminar es j_actual = j-elim
        j_actual = j - elim
        for i in range(DATA_LEN):
            cluster = data[i].get_cluster()
            if (cluster == j_actual) :
                data[i].set_cluster(None)
                is_null += 1
            elif (cluster != None and cluster > j_actual) :
                data[i].set_cluster(cluster-1)
        elim += 1
        for i in range(j_actual, num_clusters-1):
            members[i] = members[i + 1]
    
    num_clusters -= elim
    if (TALK and elim > 0) : 
        for j in range(num_clusters):
            print("El cluster ", j, " incluye ", members[j], "miembros.")
        print()

    return changed

# --------------------------
# Actualizar los clusters
KEEP_WALKING = update_clusters()

Actualizando clusters
El cluster  0  incluye  1 miembros.
El cluster  1  incluye  1 miembros.
El cluster  2  incluye  1 miembros.
El cluster  3  incluye  1 miembros.
El cluster  4  incluye  11857 miembros.

Eliminando cluster  0
Eliminando cluster  1
Eliminando cluster  2
Eliminando cluster  3
El cluster  0  incluye  11857 miembros.



In [7]:
def update_centroids():
    global centroids
    centroids = []

    if (TALK) : 
        print("Los nuevos centroids son:")
    for j in range(num_clusters):
        means = [0] * DATA_SET.shape[1]
            
        clusterSize = 0
        for k in range(len(data)):
            if(data[k].get_cluster() == j):
                p = data[k].get_value()
                for i in range(DATA_SET.shape[1]):
                    means[i] += p[i]
                clusterSize += 1

        if(clusterSize > 0):
            for i in range(DATA_SET.shape[1]):
                means[i] = means[i] / clusterSize
            centroids.append(means)

        if (TALK) : 
            print(centroids[j])        
    if (TALK) : 
        print()
    
    return

# --------------------------
# Actualizar los centroides
update_centroids()
if (elim > 0) :
    KEEP_WALKING = update_clusters()
    update_centroids()

Los nuevos centroids son:
[5932.0, 0.0077313823757582494, 0.0078350087296088847, 0.009182011044676551, 0.0080004810854549923, 0.00488751308258211, 0.0033087352598471162, 0.25217171291220375]

Actualizando clusters
El cluster  0  incluye  11861 miembros.

Los nuevos centroids son:
[5930.0, 0.0077336660483425992, 0.0078353840826628739, 0.0091820078753352859, 0.00799994085456502, 0.0048910023842235846, 0.0033076499952339813, 0.25208667060112977]



In [8]:
deltas = []
delta = 0
def update_deltas():
    global deltas, delta
    deltas = [0] * num_clusters
    delta = 0
    
    for i in range(DATA_LEN):
        cluster = data[i].get_cluster()
        deltas[cluster] += distance.euclidean(data[i].get_value(), centroids[cluster])
    mem = 0
    for i in range(num_clusters):
        delta += deltas[i]
        mem += members[i]
        deltas[i] /= members[i]
        if (TALK) : 
            print("Distancia promedio en el cluster {}:".format(i), deltas[i])        
    delta /= mem
    if (TALK) : 
        print("Distancia promedio global: {}\n".format(delta))
    
    return
    
update_deltas()

Distancia promedio en el cluster 0: 2965.2500726712483
Distancia promedio global: 2965.2500726712483



En viste de que solo quedo un cluster con todos los datos, tenemos que dividir este cluster

In [9]:
def divide_clusters():
    global num_clusters
    # Cálculo de desviaciones estandar
    sigma_vect = [[0] * DATA_SET.shape[1]] * num_clusters
    for d in range(DATA_LEN):
        cluster = data[d].get_cluster()
        p = data[d].get_value()
        for i in range(DATA_SET.shape[1]):
            sigma_vect[cluster][i] += (p[i] - centroids[cluster][i])**2        
    candidates = []
    for cluster in range(num_clusters):
        for i in range(DATA_SET.shape[1]):
            sigma_vect[cluster][i] = math.sqrt(sigma_vect[cluster][i]) / members[cluster]
            if (sigma_vect[cluster][i] > S_MAX):
                candidates.append(cluster)
                break # Sucio... pero eficiente :-) ... ya encontramos un atributo con elevada sigma
    
    divided = False
    for cluster in candidates:
        cond = num_clusters < K_INIT/2 or (deltas[cluster] > delta and members[cluster] > 2 * N_MIN)
        if(cond) :
            centroids.pop(cluster)
            points = []
            for d in range(DATA_LEN):
                if (data[d].get_cluster() == cluster):
                    points.append(data[d].get_value())
            dist = distance.squareform(distance.pdist(points, 'euclidean'))
            idx = (dist==dist.max()).argmax()
            z1 = list(points[idx // len(points)])
            z2 = list(points[idx % len(points)])
            if (TALK) :
                print("Se dividirá el cluster {}.\nSe crearán nuevos clusters en {} y {}.\n"
                     .format(cluster, z1, z2))
            centroids.append(z1)
            centroids.append(z2)
            num_clusters += 1
            divided = True
    
    if (divided) :
        if (TALK) : 
            print("Los nuevos centroids son:")
            for j in range(num_clusters):
                print(centroids[j])
            print("")

        update_clusters()
        update_centroids()
    
    return 

divide_clusters()

Se dividirá el cluster 0.
Se crearán nuevos clusters en [0.0, 0.017101840543094545, 0.0078568018884454544, 0.0091680360032164645, 0.0026941311321977261, 0.01823301610861721, 9.0654365749241037e-05, 0.0] y [11860.0, 2.2573707158255737e-06, 0.00087297798760505052, 0.0091817674600360583, 0.001347065566098863, 0.00011995405334616585, 0.37612496349360108, 1.0].

Los nuevos centroids son:
[0.0, 0.017101840543094545, 0.0078568018884454544, 0.0091680360032164645, 0.0026941311321977261, 0.01823301610861721, 9.0654365749241037e-05, 0.0]
[11860.0, 2.2573707158255737e-06, 0.00087297798760505052, 0.0091817674600360583, 0.001347065566098863, 0.00011995405334616585, 0.37612496349360108, 1.0]

Actualizando clusters
El cluster  0  incluye  5931 miembros.
El cluster  1  incluye  5930 miembros.

Los nuevos centroids son:
[2965.0, 0.011427781868527902, 0.0081031962825185728, 0.0091833309924091895, 0.00814530642336923, 0.0073610044501283773, 0.00066988853274186505, 0.004383746417130332]
[8895.5, 0.00403892

como en las siguientes iteraciones es probable que se tengan que unir cluster a parte de separarse definimos la funcion que une los clusters

In [10]:
def mix_clusters():
    global centroids, num_clusters
    dist = distance.squareform(distance.pdist(centroids, 'euclidean'))
    flag = math.floor(dist.max() * 10)
    dist[dist == 0] = flag
    
    mixed = False
    while (dist.min() < flag):
        idx = (dist==dist.min()).argmax()
        z1 = idx // len(centroids)
        z2 = idx % len(centroids)
        
        if (dist.min() < L_MIN):
            dist[z1] = flag
            dist[:,z1] = flag
            dist[z2] = flag
            dist[:,z2] = flag
            z = [sum(x)/2 for x in zip(centroids[z1], centroids[z2])]
            centroids[z1] = z
            centroids[z2] = [LARGER_DISTANCE]*DATA_SET.shape[1]
            num_clusters -= 1

            mixed = True
            if(TALK):
                print("Unificando clusters {} y {}\nSe creará nuevo centroide en {}\n"
                      .format(z1, z2, z))
        else :
            dist[z1][z2] = flag
            dist[z2][z1] = flag
        
    if (mixed) :
        update_clusters()
        update_centroids()

    return

Ahora hacemos un ciclo de separacion o union de clusters que al final nos dara clusters estables despues de que ya no se realizen cambios en ellos

In [11]:
while(iteration < I_MAX and KEEP_WALKING) :
    if (num_clusters <= K_INIT / 2) :
        divide_clusters()
    elif (iteration % 2 == 0 or num_clusters > 2 * K_INIT) :
        mix_clusters()
        
    KEEP_WALKING = update_clusters()
    if (KEEP_WALKING):
        update_centroids()
    else :
        if (TALK) : 
            print ("No más cambios.")
    iteration += 1

Se dividirá el cluster 0.
Se crearán nuevos clusters en [0.0, 0.017101840543094545, 0.0078568018884454544, 0.0091680360032164645, 0.0026941311321977261, 0.01823301610861721, 9.0654365749241037e-05, 0.0] y [5930.0, 0.0090859171311979351, 0.010475735851260605, 0.0091497273941236711, 0.001347065566098863, 0.00011995405334616585, 0.0018130873149848211, 0.0].

Los nuevos centroids son:
[8895.5, 0.0040389272743933027, 0.0075675267205455024, 0.0091806845351390092, 0.0078545507721753602, 0.0024205837918317833, 0.0059458562741616138, 0.49983136593591904]
[0.0, 0.017101840543094545, 0.0078568018884454544, 0.0091680360032164645, 0.0026941311321977261, 0.01823301610861721, 9.0654365749241037e-05, 0.0]
[5930.0, 0.0090859171311979351, 0.010475735851260605, 0.0091497273941236711, 0.001347065566098863, 0.00011995405334616585, 0.0018130873149848211, 0.0]

Actualizando clusters
El cluster  0  incluye  4448 miembros.
El cluster  1  incluye  2965 miembros.
El cluster  2  incluye  4448 miembros.

Los nuevo

In [12]:
update_deltas()

Distancia promedio en el cluster 0: 998.7505666009017
Distancia promedio en el cluster 1: 978.2499379713839
Distancia promedio en el cluster 2: 988.2500208465392
Distancia promedio global: 988.4877205344403



Despues de algunas iteraciones del metodo podemos observar que los datos quedaron divididos en 3 clusters los cuales quedaron balanceados con una cantidad de datos muy parecida en los 3.